# NLP for Text Adventure Games - part 2

In this notebook, we will introduce you word embeddings.  Word embeddings are a way of representing the meaning of words using vectors.  This style of meaning representation is different than what we saw with WordNet.  Rather than manually organizing words into a hierarchy and breaking words into their distinct senses like WordNet does, word vectors are computed automatically based on their coocurrence with other words in very large collections of texts (which are called _corpora_ in NLP).  Word vector representations do not give us hypernym, hyponym or antonym relationships by default, but they do give us an extremely straightforward way of computing the similarity of two words.  Similarity is computed by comparing the vectors and computing the angle between them with cosine.  Vectors with a very small angle between them tend to correspond to words with very similar meanings.

Starting around 2013, several algorithms were developed to create efficiently word vectors.  The most famous of which is the word2vec algorithm developed by researchers at Google. 
If you'd like to learn more about how word vectors, I recommend that you read the [Vector Semantics and Embeddings](https://web.stanford.edu/~jurafsky/slp3/6.pdf) in the [Speech and Language Processing textbook](https://web.stanford.edu/~jurafsky/slp3/) by Jurafsky and Martin who are currently updating the textbook and are relasing their draft chapters for free online while they are updating the book.

## Pre-trained Word Embeddings and Magnitude 

We are going to install a software package called Magnitude that allows for the fast efficient manipuation of word vectors.  If you'd like to learn more about it, you can read our [EMNLP 2018 paper about Magnitude](http://www.cis.upenn.edu/~ccb/publications/magnitude-fast-efficient-vector-embeddings-in-python.pdf), or you can read the [Magnitude developer documentation on Github](](https://github.com/plasticityai/magnitude)).

Then, we'll download a set of pre-trained word vectors that are stored in the Magnitude file format.  This file is several gigabytes large, so it will take a few minutes to download.

In [0]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
#!wget http://magnitude.plasticity.ai/word2vec/heavy/GoogleNews-vectors-negative300.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")
#vectors = Magnitude("GoogleNews-vectors-negative300.magnitude")

After the files have downloaded, we can start running Python and Magnitude!  We will load a set of vectors from the file that we just downloaded.<p>One the vectors are loaded, we can see how many vectors we've loaded in with this command.  This means that we have vectors representing this many words.  This is the size of our **vocabulary.**  

In [0]:
from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

print("The number of words with vector representations in this file is %s." % len(vectors))

## Word Vectors


We can see what the *dimensionality* of each vector is.  The dimensionality is just the length of the vector.  


In [0]:
vectors.dim

We can print out what a vector look likes.  It should have a bunch of real-valued numbers (positive or negative).  The number of values that we will see is *vectors.dim*

In [0]:
vectors.query("troll").shape

In [0]:
if "troll" in vectors:
  print(vectors.query("troll"))

That's what a troll looks like according to our model!  It looks just like [this](https://en.wikipedia.org/wiki/Troll_doll), right?  Well, not really, but the cool think about vectors is that they allow us to say how similar two things are.  

## Vector similarity

Having vectors for two words allows us to see how similar they are.  We can ask, how similar are *trolls* and *ogres* versus *trolls* and *princesses*?    The result will be a decimal between 0 and 1.0, with numbers closer to 1 indicating that the words are more similar.

In [0]:
print(vectors.similarity("trolls", "ogres"))
print(vectors.similarity("trolls", "princesses"))
print(vectors.similarity("princes", "princesses"))

The Magnitude software allows you to query for the most similar word out of a list of words using the command *most_similar_to_given*, which takes a query word and then a list of other words to compare it to.

In [0]:
vectors.most_similar_to_given("troll", ["princess", "prince", "ogre", "knight"]) 

## Retrieving the Most Similar Words

We can also look for the word vectors that are most similar to a query word.  Here are the words that are most similar to *trolls*.  Try replacing the word *trolls* with whatever word you want, and re-running this cell (by pressing the play button again), and see what the most similar words are to the word that you entered.

The Magnitude package uses an approximate k-nearest neighbors algorithm to efficiently retrieve the most similar vectors.  

In [0]:
vectors.most_similar_approx("trolls", topn = 20)

## Finding the most similar command
For this part of the homework, we will ask you to write a method to take in the player's command and find the command in the set of commands that your game's knows how to parser.  

You can construct a sentence embedding for a command, by taking the component wide average of words in the command.  The sentence emedding will have the same length as a word embedding.  You can compare a player's command against each of the known commands by constructing vectors for all of them, and then using Magnitude's `similarity` function

In [0]:
# FOR YOU TO DO: Write these function
def construct_sentence_vector(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    # TODO - Do something
  return sentence_vector

def find_most_similar_command(user_command, known_commands, vectors):
  # TODO - Do something
  return known_commands[0]

construct_sentence_vector("get fish", vectors)

## Solving Word Analogy Problems (Optional)

Famously, word2vec was shown to to be able solve many word analogy problems like "***man*** is to ***king*** as ***woman*** is to **-----**".  It does this by performing some vector arithmetic.   We take the vector for *king*, subtract the vector for *king*, and then add the vector for *woman*:<p>+ *king* <p>- *man*<p>+ *woman*<p>The result is a vector.  To figure out what word is closest to it, we find the most similar word vectors to the vector that resulted from our arithmetic. 

Magnitude allows us to do this in the following way:

In [0]:
vectors.most_similar(positive = ["king", "woman"], negative = ["man"])

These gender-based analogy problems also reveal that gender bias is encoded in word embeddings.  There was a paper that exposed this called [Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](http://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-d).  



## What Can You Do with a Sword? (Optional)

Nancy Fulda is one of our invited speakers for later in the term.  She and her co-authors wrote a research paper called [What Can You Do with a Rock? Affordance Extraction via Word Embeddings](https://www.ijcai.org/Proceedings/2017/0144.pdf) that developed an algorithm to extract verb associations for words using word embeddings.  

They start with a set verb/noun pairs that are simlar to text adventure commands like:

`[‘sing song’, ‘drink water’, ‘read book’, ‘eat food’, wear coat’, ‘drive car’, ‘ride horse’, ‘give gift’, ‘attack enemy’, ‘say word’, ‘open door’, ‘climb tree’, ‘heal wound’, ‘cure disease’, ‘paint picture’]`

Their algorithm can then use this list of commands and a set of pre-trained word vectors to discover verbs associated with ‘sword’, returning the following:
`[‘vanquish’, ‘duel’, ‘unsheathe’, ‘wield’, ‘behead’, ‘battle’, ‘impale’, ‘overpower’].`

If you're interested, you could reimplement their method for this week's homework.